# Preliminary instruction

To follow the code in this chapter, the `yfinance` package must be installed in your environment. If you do not have this installed yet, review Chapter 4 for instructions on how to do so.

# Chapter 11: The Long/Short Toolbox

In [ ]:
# Chapter 11: The Long/Short Toolbox

import pandas as pd
import numpy as np
import yfinance as yf
%matplotlib inline
import matplotlib.pyplot as plt

 # Convexity configuration
 We saw the risk appetite function in chapter 8. What works at the stock level for each entry also works for open risk as an aggregate

In [ ]:
# Chapter 8: Position Sizing: Money is Made in the Money Management Module

def risk_appetite(eqty, tolerance, mn, mx, span, shape):
    '''
    eqty: equity curve series
    tolerance: tolerance for drawdown (<0)
    mn: min risk
    mx: max risk
    span: exponential moving average to smoothe the risk_appetite
    shape: convex (>45 deg diagonal) = 1, concave (<diagonal) = -1, else: simple risk_appetite
    '''
    # drawdown rebased
    eqty = pd.Series(eqty)
    watermark = eqty.expanding().max() # all-time-high peak equity
    drawdown = eqty / watermark - 1 # drawdown from peak
    ddr = 1 - np.minimum(drawdown / tolerance,1) # drawdown rebased to tolerance from 0 to 1
    avg_ddr = ddr.ewm(span = span).mean() # span rebased drawdown
    
    # Shape of the curve
    if shape == 1: # 
        _power = mx/mn # convex 
    elif shape == -1 :
        _power = mn/mx # concave
    else:
        _power = 1 # raw, straight line
    ddr_power = avg_ddr ** _power # ddr 
    
    # mn + adjusted delta
    risk_appetite = mn + (mx - mn) * ddr_power 
    
    return risk_appetite



### Portfolio simulation

1. Hypothetical portfolio is benchmarked to the S&P 500 index
2. Initial capital (K) is set at USD 1 million
3. Beta (sensitivity to the market) has been extracted from the Yahoo Finance website
4. The number of shares and relative stop losses are calibrated to -0.50% relative risk adjusted to the portfolio
4. The portfolio is run from December 31, 2020, through June 30, 2021

In [ ]:
# Chapter 11: The Long/Short Toolbox

port = np.nan
K = 1000000
lot = 100
port_tickers = ['QCOM','TSLA','NFLX','DIS','PG', 'MMM','IBM','BRK-B','UPS','F']
bm_ticker= '^GSPC'
tickers_list = [bm_ticker] + port_tickers
df_data= { 
'Beta':[1.34,2,0.75,1.2,0.41,0.95,1.23,0.9,1.05,1.15],
'Shares':[-1900,-100,-400,-800,-5500,1600,1800,2800,1100,20800],
'rSL':[42.75,231,156,54.2,37.5,42.75,29.97,59.97,39.97,2.10]
}
port = pd.DataFrame(df_data,index=port_tickers)
port['Side'] = np.sign(port['Shares'])

start_dt = '2021-01-01'
end_dt = '2021-07-01'
price_df = round( yf.download(tickers= tickers_list,start= '2021-01-01' , end = '2021-07-01', 
                        interval = "1d",group_by = 'column',auto_adjust = True, 
                              prepost = True, treads = True, proxy = None)['Close'],2)

bm_cost = price_df[bm_ticker][0]
bm_price = price_df[bm_ticker][-1]

port['rCost'] = round(price_df.iloc[0,:].div(bm_cost) *1000,2)
port['rPrice'] = round(price_df.iloc[-1,:].div(bm_price) *1000,2)
port['Cost'] = price_df.iloc[0,:]
port['Price'] = price_df.iloc[-1,:]

print(port)

Let's calculate a few important measures:

1. BV, book value: cost in fund currency (USD) * open positions 
2. MV, Market Value: shares in currency (USD) : current close price 
3. rMV, relative market value 
4. Gross exposure: absolute sum of MV fx adjusted divided by K 
5. Net exposure: arithmetic net sum of MV divided by gross exposure 
6. Net beta: sum product of MV * Beta divided by gross exposure

In [ ]:
# Chapter 11: The Long/Short Toolbox

BV = port['Shares'] * port['Cost']
MV = port['Shares'] * port['Price']
rMV = port['Shares'] * port['rPrice']

port['rR'] = (port['rCost'] - port['rSL'])
port['Weight'] = round(MV.div(abs(MV).sum()),3)
port['rRisk'] = -round(np.maximum(0,(port['rR'] * port['Shares'])/K),4)
port['rRAR'] = round( (port['rPrice'] - port['rCost'])/port['rR'],1)
port['rCTR'] = round(port['Shares'] * (port['rPrice']-port['rCost'])/ K,4)
port['CTR'] = round(port['Shares'] * (port['Price']-port['Cost'])/ K,4)
port_long = port[port['Side']>0]
port_short = port[port['Side']<0]

concentration = (port_long['Side'].count()-port_short['Side'].count())/port['Side'].count()
gross = round(abs(MV).sum() / K,3) 
net = round(MV.sum()/abs(MV).sum(),3)
net_Beta = round((MV* port['Beta']).sum()/abs(MV).sum(),2)
print('Gross Exposure',gross,'Net Exposure',net,'Net Beta',net_Beta,'concentration',concentration)
rnet = round(rMV.sum()/abs(rMV).sum(),3)
rnet_Beta = round((rMV* port['Beta']).sum()/abs(rMV).sum(),2)
print('rGross Exposure',gross,'rNet Exposure',rnet,'rNet Beta',rnet_Beta)


#### Portfolio

1. R (distance from cost to stop loss) is a measure popularised by Dr Van Tharp. Here, we will be using the relative version of R, or rR
2. Weight: MV in fund currency (USD) divided by the absolute sum total of MV
3. rRisk is the weighted relative risk to the equity. As soon as the stop loss is reset beyond cost, this turns negative. This ensures that the open risk remains negative
4. rRAR is the relative returns expressed in units of initial relative risks. This is the truest and simplest risk-adjusted returns measure, our workhorse 
5. rCTR and CTR are relative and absolute contributions, or P&L divided by equity.

In [ ]:
# Chapter 11: The Long/Short Toolbox

port[['Side','Weight', 'rRisk', 'rRAR', 'rCTR', 'CTR']].sort_values(by=['Side','rRAR'] )

#### Aggregates by side

In [ ]:
# Chapter 11: The Long/Short Toolbox

print(port[['Side', 'Weight', 'rRisk', 'rRAR', 'rCTR', 'CTR']].groupby('Side').sum())

### Pro rated risk adjustment
We will pro-rate open risk by side and divide by risk-adjusted returns. This will rank positions by side and open risk-adjusted returns. Those that have contributed the least are by definition the riskiest ones. 
1. Factor the risk reduction of -1% into the pro rata to calculate the number of shares
2. Multiply by the capital and divide by the relative distance between cost and stop loss, rR, to obtain the exact number of shares
3. The risk reduction cannot be larger than the existing number of shares

In [ ]:
# Chapter 11: The Long/Short Toolbox

adjust_long = adjust_short  =  -0.01 

pro_rata_long = port_long['rRisk'] / (port_long['rRisk'].sum() * port_long['rRAR'])
risk_adj_long = (abs(adjust_long) * pro_rata_long * K / port_long['rR'] // lot) * lot
shares_adj_long =  np.minimum(risk_adj_long, port_long['Shares'])*np.sign(adjust_long)

pro_rata_short = port_short['rRisk'] / (port_short['rRisk'].sum() * port_short['rRAR'])
risk_adj_short = (abs(adjust_short) * pro_rata_short * K / port_short['rR'] // lot)*lot
shares_adj_short = np.maximum(risk_adj_short,port_short['Shares'])*np.sign(adjust_short)

port['Qty_adj'] = shares_adj_short.append(shares_adj_long)
port['Shares_adj']  = port['Shares'] + port['Qty_adj']
port['rRisk_adj'] = -round(np.maximum(0,(port['rR'] * port['Shares_adj'])/K),4)
MV_adj= port['Shares_adj'] * port['Price']
rMV_adj = port['Shares_adj'] * port['rPrice']
port['Weight_adj'] = round(MV_adj.div(abs(MV_adj).sum()),3)

print(port[['Side','rRAR','rRisk','rRisk_adj','Shares','Qty_adj', 'Shares_adj', 'Weight','Weight_adj']].groupby('Side').sum())


#### Aggregates

In [ ]:
# Chapter 11: The Long/Short Toolbox

port[['Side','rRAR','rRisk','rRisk_adj','Shares','Qty_adj', 'Shares_adj', 'Weight','Weight_adj']].groupby('Side').sum()

#### Portfolio before and after adjustment

In [ ]:
# Chapter 11: The Long/Short Toolbox

port[['Side','rRAR','rRisk','rRisk_adj','Shares','Qty_adj', 'Shares_adj', 'Weight','Weight_adj']].sort_values(
        by=['Side','rRisk_adj' ], ascending=[True,False])

#### Aggregates before and after adjustment

In [ ]:
# Chapter 11: The Long/Short Toolbox

print('Gross Exposure',gross,'Net Exposure',net,'Net Beta',net_Beta,'concentration',concentration)
gross_adj = round(abs(MV_adj).sum() / K,3) 
net_adj = round(MV_adj.sum()/abs(MV_adj).sum(),3)
net_Beta_adj = round((MV_adj* port['Beta']).sum()/abs(MV_adj).sum(),2)
net_pos_adj = port.loc[port['Shares_adj'] >0,'Shares_adj'].count()-port.loc[port['Shares_adj'] <0,'Shares_adj'].count()
print('Gross Exposure adj',gross_adj,'Net Exposure_adj',net_adj,
      'Net Beta_adj',net_Beta_adj,'concentration adj',net_pos_adj)
rnet_adj = round(rMV_adj.sum()/abs(rMV_adj).sum(),3)
rnet_Beta_adj = round((rMV_adj* port['Beta']).sum()/abs(rMV_adj).sum(),2)
print('Gross Exposure adj',gross_adj,'rNet Exposure adj',rnet_adj,'rNet Beta adj',rnet_Beta_adj)
